In [1]:
#DATABASE ENGINEERING
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base as declarative_base_HM
from sqlalchemy.ext.declarative import declarative_base as declarative_base_HS
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date
from sqlalchemy.orm import * 
import datetime

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Create an engine to database 
engine_HM = create_engine("sqlite:///cleaned_Hawaii_Measurements.sqlite")
#engine_HS = create_engine("sqlite:///cleaned_Hawaii_Stations.sqlite")

In [4]:
conn_HM = engine_HM.connect()
#conn_HS = engine_HS.connect()

In [5]:
Base_HM = declarative_base_HM()

In [6]:
class Measurement(Base_HM):
    __tablename__ = 'Measurement'

    m_id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    

    def __repr__(self):
        return f"id={self.m_id}, station={self.station}"

# class Station(Base):
#     __tablename__ = 'Station'

#     s_id = Column(Integer, primary_key=True)
#     station = Column(Text)
#     name = Column(Text)
#     latitude = Column(Float)
#     longitude = Column(Float)
#     elevation = Column(Float)
    

#     def __repr__(self):
#         return f"id={self.s_id}, station={self.station}"

In [7]:
Base_HM.metadata.create_all(engine_HM)

#Base.metadata.create_all(engine_HS)

In [8]:
inspector = inspect(engine_HM)
inspector.get_table_names()

['Measurement']

In [9]:
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

m_id INTEGER
station TEXT
date DATE
prcp FLOAT
tobs FLOAT


In [10]:
# CSV file
csvfile = "cleaned_Hawaii_Measurements.csv"
csvfile_2 = "cleaned_Hawaii_Stations.csv"

In [11]:
df_HM = pd.read_csv(csvfile)
df_HS = pd.read_csv(csvfile_2)

In [12]:
df_HM['date'].values[0]

'2010-01-01'

In [15]:
#datetime.strptime(dataframe_HM['date'],'%Y-%m-%d')
date = [i for i in df_HM['date']]
dates = []
for i in date:
    dates.append((datetime.datetime.strptime(i,'%Y-%m-%d')).date())

In [16]:
df_HM.drop(['date'], axis = 1, inplace = True)


In [17]:
df_HM['date'] = dates

In [18]:
df_HM['date'].values[0]

datetime.date(2010, 1, 1)

In [19]:
data_HM = df_HM.to_dict(orient='records')
data_HM[0]

{'date': datetime.date(2010, 1, 1),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [20]:
# Use MetaData from SQLAlchemy to reflect the tables
### BEGIN SOLUTION
metadata = MetaData(bind=engine_HM)
metadata.reflect()

In [21]:
# Save the reference to the `demographics` table as a variable called `table`
### BEGIN SOLUTION
table_HM = sqlalchemy.Table('Measurement', metadata, autoload=True)
### END SOLUTION

In [22]:
# Use `table.insert()` to insert the data into the table
### BEGIN SOLUTION
conn_HM.execute(table_HM.insert(), data_HM)
### END SOLUTION

In [24]:
conn_HM.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]